In [15]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle
import io

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [16]:
from functions import replace_jalalidate
from functions import find_month

# **Loading Data**

In [17]:
%%time
measures = pd.DataFrame()
balance  = pd.DataFrame()
income   = pd.DataFrame()
cashflow = pd.DataFrame()
dividend = pd.DataFrame()

for year in range(1387,1399):
    # FileNames
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    file_name_measures = "Measures"+str(year)+".csv"
    file_name_balance  = "BalanceSheet"+str(year)+".csv"
    file_name_income   = "IncomeStatement"+str(year)+".csv"
    file_name_cashflow = "CashFlow"+str(year)+".csv"
    file_name_dividend = "Dividend"+str(year)+".csv"
    
    # reading and appending Data
    try:
        temp = pd.read_csv(file_name_measures,index_col=0).drop_duplicates()
        temp['Year'] = year
        measures = measures.append(temp)
    except:
        print('measures_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_balance,index_col=0).drop_duplicates()
        balance = balance.append(temp)
    except:
        print('balance_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_income,index_col=0).drop_duplicates()
        income = income.append(temp)
    except:
        print('income_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_cashflow,index_col=0).drop_duplicates()
        cashflow = cashflow.append(temp)
    except:
        print('cashflow_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_dividend,index_col=0).drop_duplicates()
        dividend = dividend.append(temp)
    except:
        print('dividend_'+str(year))
        

dividend_1398
Wall time: 2.94 s


# Merging with Balancesheet Data

In [18]:
balance_star = balance[['Firm', 'Symbol','Fin_year', 'Year', 'PPE','Tot_asset', 'Tot_lib','Book_value','Capital','Equity']]
balance_star.rename(columns={'Capital':'Capital_blnc','Equity':'Equity_blnc','Fin_year':'Fin_year_blnc'},inplace=True)

len(balance) == len(balance.drop_duplicates(['Symbol','Year']))
# balance_star.head()

Merged_l1 = pd.merge(measures,balance_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


True

# Merging with Income Statement Data

In [19]:
income_star = income[['Symbol','Fin_year', 'Year', 'Revenue','Cost_of_revenue', 'Gross_profit',
                      'Operating_expenses','Operating_income', 'EBT',
                      'Tax', 'Net_income',
                      'EPS', 'Capital', 'Marketcap']]

income_star.rename(columns={'Capital':'Capital_inc','Marketcap':'Marketcap_inc','EPS':'EPS_inc',
                           'Fin_year':'Fin_year_inc'},inplace=True)
len(income) == len(income.drop_duplicates(['Symbol','Year']))
# income_star.head()

Merged_l2 = pd.merge(Merged_l1,income_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

True

# Merging with Cashflow Statement Data

In [20]:
cashflow_star = cashflow[['Symbol','Year','Fin_year',
                          'Operating_cash_flow','Investing_cash_flow', 'Financing_cash_flow',
                          'Begining_cash_position','Change_in_cash', 'End_cash_position']]
cashflow_star.rename(columns={'Fin_year':'Fin_year_cf'},inplace=True)
len(cashflow) == len(cashflow.drop_duplicates(['Symbol','Year']))
# cashflow_star.head()

Merged_l3 = pd.merge(Merged_l2,cashflow_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

True

# Merging with Dividend Data

In [21]:
dividend_star = dividend[['Year','Symbol', 'Earning', 'Dividend', 'Payout',
                           'EPS', 'DPS', 'Equity', 'adj_eps', 'adj_dps', 'adj_earning',
                           'adj_dividend']]

dividend_star.rename(columns={'Earning':'Earning_div','EPS':'EPS_div',
                             'Equity':'Equity_div'},inplace=True)
len(dividend) == len(dividend.drop_duplicates(['Symbol','Year']))
set(dividend[dividend.duplicated(['Symbol','Year'],keep=False)].Firm)
# dividend_star.head()
Merged = pd.merge(Merged_l3,dividend_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

False

{'سرمایه گذاری نور کوثر ایرانیان',
 'نهادهای مالی بورس اوراق بهادار',
 'کارگزاران بورس اوراق بهادار'}

## **Preparing Data**

### **Unifying Data and Filling NAs**

In [22]:
Merged.Equity_blnc.fillna(Merged.Equity_div,inplace=True)
Merged.Net_income.fillna(Merged.Earning_div,inplace=True)
Merged.EPS_inc.fillna(Merged.EPS_div,inplace=True)
Merged.Capital_blnc.fillna(Merged.Capital_inc,inplace=True)
Merged.Marketcap_inc.fillna(Merged.MarketCap,inplace=True)
Merged.Fin_year_inc.fillna(Merged.Fin_year_cf,inplace=True)
Merged.Fin_year_blnc.fillna(Merged.Fin_year_inc,inplace=True)

Merged.rename(columns={'Equity_blnc':'Equity','EPS_inc':'EPS','Capital_blnc':'Capital',
                      'Marketcap_inc':'Marketcap','Fin_year_blnc':'Fin_year'},inplace=True)

Merged['Fin_year'] = Merged['Fin_year'].apply(lambda x: replace_jalalidate(x))
Merged['Month'] = Merged['Fin_year'].apply(lambda x: find_month(x))

Merged.drop(columns=['Equity_div','Earning_div','EPS_div','Capital_inc','MarketCap'],inplace=True)


# Finding Industry Id
Industry_list = Merged.Industry.drop_duplicates().sort_values()
Industry_dict = dict(zip(Industry_list,range(len(Industry_list))))
Merged['Industry_id'] = Merged['Industry'].map(Industry_dict)

In [23]:
# Selecting Columns and sorting
Data = Merged[['Id_tse','Firm','Symbol','Fin_year','Year','Month', 'Industry','Industry_id', 'sum_over1', 'Num_holders',
       'Largest_Owner', 'First_Second', 'First_Sumtwofour', 'Sumfive', 'Gini',
       'Herfindhal', 'SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL',
       'Tot_asset', 'Tot_lib','PPE', 'Book_value','Marketcap','Capital','Equity',
       'Revenue', 'Cost_of_revenue', 'Gross_profit','Operating_expenses',
       'Operating_income', 'EBT', 'Tax', 'Net_income','EPS',
       'Operating_cash_flow', 'Investing_cash_flow','Financing_cash_flow',
       'Begining_cash_position', 'Change_in_cash','End_cash_position',
       'Dividend', 'Payout', 'DPS', 'adj_eps', 'adj_dps','adj_earning', 'adj_dividend']]

In [24]:
# Completing Firms
temp = Data[['Symbol','Firm']].sort_values(by='Symbol').dropna().drop_duplicates('Symbol')
sym_firm_dict = {k:v for (k,v) in zip(temp['Symbol'],temp['Firm'])}
Data['Firm'] = Data['Symbol'].map(sym_firm_dict)

C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [25]:
Data.head()

,Id_tse,Firm,Symbol,Fin_year,Year,Month,Industry,Industry_id,sum_over1,Num_holders,...,Begining_cash_position,Change_in_cash,End_cash_position,Dividend,Payout,DPS,adj_eps,adj_dps,adj_earning,adj_dividend
0,44834847569322522,کنتورسازی ایران,آکنتور,NaN,1387,NaN,ابزارپزشکی، اپتیکی و اندازه‌گیری,0,2.12,1,...,NaN,NaN,NaN,9450.0,0.8523,900.0,3960.990248,3375.843961,41590.397599,35446.361590
1,62952165421099192,کابل البرز,بالبر,NaN,1387,NaN,ماشین آلات و دستگاه‌های برقی,37,63.24,9,...,NaN,NaN,NaN,0.0,0.0000,0.0,93.773443,0.000000,1069.017254,0.000000
2,46752599569017089,ایران ترانسفو,بترانس,NaN,1387,NaN,ماشین آلات و دستگاه‌های برقی,37,59.26,6,...,NaN,NaN,NaN,22500.0,0.4028,750.0,6984.246062,2813.203301,209527.381845,84396.099025
3,47377315952751604,پارس سوئیچ,بسویچ,NaN,1387,NaN,ماشین آلات و دستگاه‌های برقی,37,93.44,5,...,NaN,NaN,NaN,5800.0,0.8180,400.0,1834.208552,1500.375094,26596.024006,21755.438860
4,69454539056549106,لامپ پارس شهاب,بشهاب,NaN,1387,NaN,ماشین آلات و دستگاه‌های برقی,37,96.63,3,...,NaN,NaN,NaN,1890.0,0.5618,350.0,2336.834209,1312.828207,12618.904726,7089.272318


### **Adjustments to Data**

In [26]:
# Renaming
Data.rename(columns={'Herfindhal':'Herfindahl'},inplace=True)

# Unit conversion
Data['Herfindahl'] = Data['Herfindahl']*100
Data['Marketcap'] = Data['Marketcap']/1e6
Data['Dividend'] = Data['Dividend']*10

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

### **Filtering Data**

In [27]:
## Filtering Data
print('Number of Observation is: ',len(Data),'\n')
theta_sum1     = 95
theta_holder   = 1
MC             = 150e4    # X bilion Toman!
Balancing      = False
Balancing_year = 1390
Banned_Industries = ['واسطه‌گری‌های مالی و پولی','بیمه وصندوق بازنشستگی به جزتامین اجتماعی','بانکها و موسسات اعتباری','سرمایه گذاریها','سایر واسطه گریهای مالی',
                              'صندوق سرمایه گذاری قابل معامله','فعالیتهای کمکی به نهادهای مالی واسط']

# Orginal Data
df1 = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Orginal'})

# Removing financial firms
Data = Data[~Data.Industry.isin(Banned_Industries)]
df2 = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Removing Financial Firms'})
out = pd.merge(df1,df2,left_on='Year',right_on='Year',how='left')

# Removing by MarketCap
temp = Data[Data.Year==1398]
ls = list(temp[temp.Marketcap<MC]['Symbol'])
Data = Data[~Data.Symbol.isin(ls)]
df3 = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Removing Based on MarketCap'})
out = pd.merge(out,df3,left_on='Year',right_on='Year',how='left')


# Removing Firms with just 1 holder
Data = Data[Data.Num_holders>theta_holder]
df4 = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Removing Based on Number of Holders'})
out = pd.merge(out,df4,left_on='Year',right_on='Year',how='left')

# Removing whit sum_over1
Data = Data[Data.sum_over1<=theta_sum1]
df5 = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Removing Based on sum_over1'})
out = pd.merge(out,df5,left_on='Year',right_on='Year',how='left')

# Removing with Fin_year
temp = Data[pd.isnull(Data.Month)]
Data = Data[Data.Month==12]
df6 = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Removing Based on Fin_year'})
df6_prime = temp.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Missing Fin_year(Month)'})
df6['Missing Fin_year(Month)'] = df6_prime['Missing Fin_year(Month)']
df6 = df6[['Year','Missing Fin_year(Month)','Removing Based on Fin_year']]
out = pd.merge(out,df6,left_on='Year',right_on='Year',how='left')

print('Number of remained Observation before balancing is: ',len(Data),'\n')

# Balancing the panel data: keeping common stocks in the year x and after that
Year_Symbol_dict=dict()
for year in Data.Year.drop_duplicates().sort_values():
    temp = Data[Data.Year==year]
    Year_Symbol_dict[str(year)] = set(temp.Symbol)
    
# Years 1389-97
df7 = pd.DataFrame()
common_sym_dict = dict()

for year in range(1387,1397):
    common_sym = Year_Symbol_dict[str(year)]
    for i in range(year+1,1398):
        common_sym = common_sym.intersection(Year_Symbol_dict[str(i)])
    df7 = df7.append(pd.DataFrame(data={'Year':[year],'Remaining Symbols':[len(common_sym)]}))
    common_sym_dict[str(year)] = common_sym

df7.rename(columns={'Remaining Symbols':'Balancing in the year x to year 1397'},inplace=True)    
out = pd.merge(out,df7,left_on='Year',right_on='Year',how='left')

if Balancing:
    Data = Data[Data.Year>=Balancing_year] 
    Data = Data[Data.Year<=1397]
    Data = Data[Data.Symbol.isin(list(common_sym_dict[str(Balancing_year)]))]
    print('Number of remained Observation After balancing with year',str(Balancing_year),' is: ',len(Data),'\n')
    
# Filterring By time
Data = Data[Data.Year>=1389] 
Data = Data[Data.Year<=1397]


print('Number of remained Observation after balancing and filltering by date: ',len(Data),'\n')
out.fillna(0)


Number of Observation is:  4611 

Number of remained Observation before balancing is:  1769 

Number of remained Observation after balancing and filltering by date:  1678 



,Year,Orginal,Removing Financial Firms,Removing Based on MarketCap,Removing Based on Number of Holders,Removing Based on sum_over1,Missing Fin_year(Month),Removing Based on Fin_year,Balancing in the year x to year 1397
0,1387,238,216,216,208,192,189,2,1.0
1,1388,242,215,215,207,198,122,47,17.0
2,1389,264,230,230,219,205,2,159,84.0
3,1390,282,244,244,231,199,1,150,93.0
4,1391,301,257,257,239,204,3,152,101.0
5,1392,308,265,265,249,216,3,161,113.0
6,1393,349,290,290,266,223,1,165,126.0
7,1394,362,290,290,270,229,3,172,148.0
8,1395,413,324,324,300,262,2,195,173.0
9,1396,592,454,452,410,361,10,259,235.0


## **Saving Data**

In [28]:
Data.to_csv('Merged_Data_July.csv')